In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [2]:
df = pd.read_csv('Churn_Modelling.csv')

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [88]:
print(df['Gender'].value_counts())
label = LabelEncoder()
df['Gender'] = label.fit_transform(df['Gender'])


Gender
Male      5457
Female    4543
Name: count, dtype: int64


In [89]:
df['Geography'].value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [90]:
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder()
df_geo = pd.DataFrame(df['Geography'])
df_geo = onehot.fit_transform(df_geo).toarray()
df_geo = pd.DataFrame(df_geo, columns=onehot.get_feature_names_out())
df_geo


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [91]:
df= pd.concat([df,df_geo],axis=1)
df.drop(['Geography'], axis=1, inplace=True)
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,1,15634602,Hargrave,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,2,15647311,Hill,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,3,15619304,Onio,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,4,15701354,Boni,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,5,15737888,Mitchell,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [92]:
df.drop(['RowNumber','CustomerId','Surname'],inplace=True,axis=1)

In [53]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [54]:
df.isnull().sum()

CreditScore          0
Gender               0
Age                  0
Tenure               0
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
Geography_France     0
Geography_Germany    0
Geography_Spain      0
dtype: int64

In [59]:
for col in df.columns:
    cnt = 0
    #q2 = df[col].median()
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    IQR = q3-q1
    maxi = q3+(1.5*IQR)
    mini = q1-(1.5*IQR)
    for val in df[col]:
        if val>maxi or val<mini:
            cnt = cnt+1
    print(f"Outliers in Column {col} : {cnt}")


Outliers in Column CreditScore : 15
Outliers in Column Gender : 0
Outliers in Column Age : 359
Outliers in Column Tenure : 0
Outliers in Column Balance : 0
Outliers in Column NumOfProducts : 60
Outliers in Column HasCrCard : 0
Outliers in Column IsActiveMember : 0
Outliers in Column EstimatedSalary : 0
Outliers in Column Exited : 2037
Outliers in Column Geography_France : 0
Outliers in Column Geography_Germany : 0
Outliers in Column Geography_Spain : 2477


In [78]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [93]:
## Save the encoder and labels
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label,file)

with open('onehot_encoder_geography','wb') as file:
    pickle.dump(onehot,file)

In [94]:
## Divide the dataset in indepenedent and dependent features
X = df.drop(['Exited'], axis=1)
y = df['Exited']

In [103]:
## SPlit the data into train and test 
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [104]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [106]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)


In [108]:
X_train.shape

(8000, 12)

ANN Implementation

In [107]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [109]:
## Building ANN Model
model = Sequential(
    [
        Dense(64,activation='relu', input_shape=(X_train.shape[1],)), #first hidden layer 1
        Dense(32, activation='relu'), #HL2
        Dense(1,activation='sigmoid') ##output layer
    ]
)

c:\Users\yashs\OneDrive\Desktop\ML\ETE_DL_PROJECT_ChurnModeling\venvdl\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [110]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [112]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
# this is done to give our own learning rate, we can also directly write the optimizer name
loss = tensorflow.keras.losses.BinaryCrossentropy()

In [114]:
 ## Compile the model
model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["accuracy"])

In [123]:
## setup the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
log_dir = "log/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir= log_dir, histogram_freq=1)

In [124]:
## Setting up Early Stoping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [125]:
## Training the model
history = model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8646 - loss: 0.3367 - val_accuracy: 0.8580 - val_loss: 0.3443
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8645 - loss: 0.3229 - val_accuracy: 0.8575 - val_loss: 0.3445
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8637 - loss: 0.3227 - val_accuracy: 0.8515 - val_loss: 0.3637
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8639 - loss: 0.3263 - val_accuracy: 0.8640 - val_loss: 0.3413
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8700 - loss: 0.3240 - val_accuracy: 0.8585 - val_loss: 0.3431
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8609 - loss: 0.3268 - val_accuracy: 0.8565 - val_loss: 0.3536
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8677 - loss: 0.3187 - val_accuracy: 0.8605 - val_loss: 0.3485
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8738 - loss: 0.3105 - val_accu

In [126]:
model.save('model.h5')

In [127]:
## Load tensorboard extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [128]:
%tensorboard --logdir log/fit

Reusing TensorBoard on port 6007 (pid 22820), started 0:04:33 ago. (Use '!kill 22820' to kill it.)

In [ ]:
## Load the pickle file
